In [ ]:
#!pip install gradio==3.30.0

In [ ]:
import hopsworks

project = hopsworks.login()

In [ ]:
ms = project.get_model_serving()

# get query deployment metadata object
query_model_deployment =  ms.get_deployment("querydeployment")

## <span style="color:#ff5f27">🔮 Get ranking of recommendations by customer </span>


In [ ]:
import gradio as gr
from datetime import datetime

def parse_output(ranking):
    output = ""
    for rec in ranking:
        output += "Item ID: " + str(rec[1]) + "\t\t--\t\t score: " + str(rec[0]) + "\n"
    return output

def recommender(c_id, dt):
    list_inputs = []
    list_inputs.append(c_id)
    list_inputs.append(dt)    
    mop = datetime.now().isoformat() if dt == "now" else dt
    data = {"instances": {"customer_id": c_id, "transaction_date": mop}}
    res = query_model_deployment.predict(data)
    return parse_output(res["predictions"]["ranking"])

demo = gr.Interface(
    fn=recommender,
    title="Fashion Items Recommender",
    description="Get recommendations of fashion items based on customer history and item features.",
    allow_flagging="never",
    inputs=[
        gr.inputs.Textbox(default="641e6f3ef3a2d537140aaa0a06055ae328a0dddf2c2c0dd6e60eb0563c7cbba0", label="For which customer?"),
        gr.inputs.Textbox(default="now", label="When to purchase?"),
    ],
    outputs=gr.outputs.Textbox(label="Recommendations")
)

demo.launch(share=True)